In [1]:
#import warnings
#warnings.filterwarnings('ignore')
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold

# Loading 
## Load Model

In [19]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

## Load Dataset

In [3]:
# load prepocessed data
df=pd.read_csv('/raw_data/backstory.csv')


# Split the dataframe into train and test sets, with a test size of 0.03
train, test = train_test_split(df, test_size=0.03, random_state=42)

# Create a cross-validation iterator with 5 folds
kf = KFold(n_splits=5, shuffle=True, random_state=42)



FileNotFoundError: [Errno 2] No such file or directory: 'processed_data.csv'

# Hyperparameters

In [5]:
# Add a new special token "<Name_def>"
token_name = "[PAWN_nameDef]"
token_possessive ="[PAWN_possessive]"
token_pronoun ="[PAWN_pronoun]"

# Model Training

In [20]:
# Add custom tokens to tokenizer
#tokenizer.add_tokens([token_name,token_possessive,token_pronoun])
model.resize_token_embeddings(len(tokenizer))

# five fold cross validation
# Loop over the train set and the cross-validation iterator
"""for train_index, val_index in kf.split(train):
    # Get the train and validation sets for each fold
    train_fold = train.iloc[train_index]
    val_fold = train.iloc[val_index]
    # Training process"""

train_fold=pd.DataFrame(columns=["title", "desc", "attributes"])
train_fold.at[0, 'title'] = "dummy one"
train_fold.at[0, 'desc'] = "This is a description of the title dummy one, used for testing purposes"
#random story generation
for i in range (20):
    prompt = "This is a story generated based on the number "+str(i)
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    outputs = model.generate(input_ids, do_sample=True, max_length=30, temperature =11.6+0.75*i,repetition_penalty=1.1)
    train_fold.loc[i+1] = ["dummy "+str(i), tokenizer.batch_decode(outputs, skip_special_tokens=True), None ]

train_fold




You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embeding dimension will be 50257. This might induce some performance reduction as *Tensor Cores* will not be available. For more details  about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not 

,title,desc,attributes
0,dummy one,"This is a description of the title dummy one, ...",NaN
1,dummy 0,[This is a story generated based on the number...,None
2,dummy 1,[This is a story generated based on the number...,None
3,dummy 2,[This is a story generated based on the number...,None
4,dummy 3,[This is a story generated based on the number...,None
5,dummy 4,[This is a story generated based on the number...,None
6,dummy 5,[This is a story generated based on the number...,None
7,dummy 6,[This is a story generated based on the number...,None
8,dummy 7,[This is a story generated based on the number...,None
9,dummy 8,[This is a story generated based on the number...,None


In [ ]:

# Save trained model
#torch.save(model.state_dict(), 'models/model_v1.pt')

# Output generation

In [ ]:
prompt = "This is the story of Glouse, a tribal archer, from glitter world"
prompt = "This is a story generated based on the number"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
# sample up to 30 tokens
torch.manual_seed(11)
list_out=[]
#['Today I believe we can finally get rid of discrimination," said Rep. Mark Pocan (D-Wis.).\n\n"Just look at the']
for i in range (1):
    for j in range (8):
        outputs = model.generate(input_ids, do_sample=True, max_length=30, temperature =11.6+0.75*i,top_p=(j+1)/30,repetition_penalty=1.1)
        list_out.append(outputs)

for i in range (1):
    for j in range (8):
        print("temperature =",11.6+0.75*i,"top_p=",(j+1)/30)
        print(tokenizer.batch_decode(list_out[i*4+j], skip_special_tokens=True))

# Note Area

1. asn3 mingpt
2. asn3 hugging face

# Testing Area

In [33]:
# Import the necessary modules
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
import torch
from datasets import load_dataset

# Load the dataset
#dataset = load_dataset("wikitext", "wikitext-2-raw-v1")
dataset = train_dataset

# Preprocess the dataset
def encode(examples):
    input_ids = tokenizer(examples["text"], return_tensors="pt", add_special_tokens=False).input_ids
    labels = input_ids.clone()
    return {"input_ids": input_ids, "labels": labels}

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")
dataset = dataset.map(encode, batched=True, remove_columns=["text"])

# Define the training arguments
training_args = TrainingArguments(
    output_dir="output",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    learning_rate=5e-5,
    logging_steps=100,
    save_steps=500,
    save_total_limit=3,
)

# Create a trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
)

# Train the model
trainer.train()

AttributeError: 'LanguageModelingDataset' object has no attribute 'map'

In [28]:
import torch
import numpy as np

from nltk.tokenize import sent_tokenize

from pathlib import Path
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
#from mingpt.bpe import BPETokenizer
#from mingpt.utils import set_seed
#set_seed(1234)
"""
Prepare the dataset to train the Language Model (LM)
This implementation splits the sentences and so doesn't create training 
examples that cross sentences.

This code is set so that it uses one of two possible datasets, which were also used in Assignment 1: 
SmallSimpleCorpus.txt or LargerCorpus.txt

Arguments:
            ds_choice: str. "small" or "large". (i.e. selects which of the two datasets)
            split: str. "train" or "test".
            truncation: int. If -1: no truncation on sentences. Otherwise: truncate to this specific length.
"""

class LanguageModelingDataset(Dataset):

    def __init__(self, ds_choice="small", split="train", truncation=-1):

        base_path = "./"
        fn = {"small": "raw_data/backstory.csv", "large": "LargerCorpus.txt"}
        self.ds_choice = ds_choice
        self.truncation = truncation  # int. If -1, then
        text = Path(base_path, fn[ds_choice]).read_text()#encoding='utf-8')
        if ds_choice == "large":
            # Remove the newline char in the middle of sentences
            # The "paragraph splitting" newlines appear to be \n\n -- remove the duplications there
            text = text.replace("\n\n", "$$^^$$").replace("\n", " ").replace("$$^^$$", "\n")
        sentences = sent_tokenize(text)
        for i in range(10):
            print(sentences[i])

        # Train / test split
        train, val = train_test_split(sentences, test_size=0.2, shuffle=False)
        if split == "train":
            raw_data = train
        else:
            raw_data = val

            # Tokenize
        self.tokenizer = tokenizer
        self.data = []  # List of 1-d pytorch tensor
        for sent in raw_data:
            tokenized = self.tokenizer(sent).view(-1)  # pytorch tensor
            if truncation >= 0:
                self.data.append(tokenized[:truncation])
            else:
                self.data.append(tokenized)

        # Count some items
        self.max_sentence_length = np.max([len(d) for d in self.data])

    def __len__(self):
        return len(self.data)

    def get_vocab_size(self):
        """
        We have to set this to the max vocab size (i.e., that decided by the BPE tokenizer), 
        but actually, only a small number of vocab is used, especially for the small text. 
        """
        return 50257

    def __getitem__(self, idx):
        """
        The output should be a tuple x and y, both as pytorch tensors.
        Please refer to the `run()` method in the mingpt/trainer.py script for 
        how the x and y are going to be used.
        """
        x = self.data[idx][:-1]
        y = self.data[idx][1:]
        return (x, y)

    def get_block_size(self):
        """
        block_size is the size at which lines are truncated to ensure they are equal-length.
        """
        return self.max_sentence_length

# Instantiate the Training Dataset
#train_dataset = LanguageModelingDataset(ds_choice="small", split="train")  # use this for the short corpus
train_dataset = LanguageModelingDataset(ds_choice="small", split="train", truncation=512) #use this for long

# Instantiate a Validation Dataset (this is only really needed for the fine-tune task, not the LM task)
#val_dataset = LanguageModelingDataset(ds_choice="small", split="validation")
#val_dataset = LanguageModelingDataset(ds_choice="large", split="validation", truncation=512)

train_dataset

Title,Name,Desc,Attribute

sewer kid,SewerKid57,"[PAWN_nameDef] grew up in the sludge-smeared sewers of a polluted industrial world.
\n[PAWN_pronoun] befriended the strange insects in the darkness, and learned to love them.
",Cooking+2	

navy scientist,NavyScientist52,"Interstellar warfare is won by technology, so glitterworld navies are always on the peak of modern research.
Even better, they have first access to archotechnological artifacts because they find them in space.\n\n[PAWN_nameDef] worked in a navy lab.
",Intellectual+8	

glitterworld empath,GlitterworldEmpath26,"[PAWN_nameDef] had an amazing ability to relate to others and mediate conflict.
[PAWN_pronoun] was selected to train as an empath.
The training intensified [PAWN_possessive] natural abilities, but [PAWN_pronoun] became unable to harm others.
",Social+8	Medicine+2	

glitterworld surgeon,GlitterworldSurgeon15,"[PAWN_nameDef] worked as a surgeon on a world mostly free of disease and human suffering.\n\n[PAWN_possessive

AttributeError: 

## fix tokenize

In [32]:
import torch
import numpy as np

from nltk.tokenize import sent_tokenize

from pathlib import Path
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
#from mingpt.bpe import BPETokenizer
#from mingpt.utils import set_seed
#set_seed(1234)
"""
Prepare the dataset to train the Language Model (LM)
This implementation splits the sentences and so doesn't create training 
examples that cross sentences.

This code is set so that it uses one of two possible datasets, which were also used in Assignment 1: 
SmallSimpleCorpus.txt or LargerCorpus.txt

Arguments:
            ds_choice: str. "small" or "large". (i.e. selects which of the two datasets)
            split: str. "train" or "test".
            truncation: int. If -1: no truncation on sentences. Otherwise: truncate to this specific length.
"""

class LanguageModelingDataset(Dataset):

    def __init__(self, ds_choice="small", split="train", truncation=-1):

        base_path = "./"
        fn = {"small": "raw_data/backstory.csv", "large": "LargerCorpus.txt"}
        self.ds_choice = ds_choice
        self.truncation = truncation  # int. If -1, then
        text = Path(base_path, fn[ds_choice]).read_text()#encoding='utf-8')
        if ds_choice == "large":
            # Remove the newline char in the middle of sentences
            # The "paragraph splitting" newlines appear to be \n\n -- remove the duplications there
            text = text.replace("\n\n", "$$^^$$").replace("\n", " ").replace("$$^^$$", "\n")
        sentences = sent_tokenize(text)
        for i in range(10):
            print(sentences[i])

        # Train / test split
        train, val = train_test_split(sentences, test_size=0.2, shuffle=False)
        if split == "train":
            raw_data = train
        else:
            raw_data = val

            # Tokenize
        self.tokenizer = tokenizer
        self.data = []  # List of 1-d pytorch tensor
        
        for sent in raw_data:
            tokenized = self.tokenizer(sent)
            print(sent)
            print(tokenized)
            #tokenized = self.tokenizer(sent).view(-1)  # pytorch tensor
            if truncation >= 0:
                self.data.append(tokenized[:truncation])
            else:
                self.data.append(tokenized)

        # Count some items
        self.max_sentence_length = np.max([len(d) for d in self.data])

    def __len__(self):
        return len(self.data)

    def get_vocab_size(self):
        """
        We have to set this to the max vocab size (i.e., that decided by the BPE tokenizer), 
        but actually, only a small number of vocab is used, especially for the small text. 
        """
        return 50257

    def __getitem__(self, idx):
        """
        The output should be a tuple x and y, both as pytorch tensors.
        Please refer to the `run()` method in the mingpt/trainer.py script for 
        how the x and y are going to be used.
        """
        x = self.data[idx][:-1]
        y = self.data[idx][1:]
        return (x, y)

    def get_block_size(self):
        """
        block_size is the size at which lines are truncated to ensure they are equal-length.
        """
        return self.max_sentence_length

# Instantiate the Training Dataset
#train_dataset = LanguageModelingDataset(ds_choice="small", split="train")  # use this for the short corpus
train_dataset = LanguageModelingDataset(ds_choice="small", split="train", truncation=512) #use this for long

# Instantiate a Validation Dataset (this is only really needed for the fine-tune task, not the LM task)
#val_dataset = LanguageModelingDataset(ds_choice="small", split="validation")
#val_dataset = LanguageModelingDataset(ds_choice="large", split="validation", truncation=512)

train_dataset

Title,Name,Desc,Attribute

sewer kid,SewerKid57,"[PAWN_nameDef] grew up in the sludge-smeared sewers of a polluted industrial world.
\n[PAWN_pronoun] befriended the strange insects in the darkness, and learned to love them.
",Cooking+2	

navy scientist,NavyScientist52,"Interstellar warfare is won by technology, so glitterworld navies are always on the peak of modern research.
Even better, they have first access to archotechnological artifacts because they find them in space.\n\n[PAWN_nameDef] worked in a navy lab.
",Intellectual+8	

glitterworld empath,GlitterworldEmpath26,"[PAWN_nameDef] had an amazing ability to relate to others and mediate conflict.
[PAWN_pronoun] was selected to train as an empath.
The training intensified [PAWN_possessive] natural abilities, but [PAWN_pronoun] became unable to harm others.
",Social+8	Medicine+2	

glitterworld surgeon,GlitterworldSurgeon15,"[PAWN_nameDef] worked as a surgeon on a world mostly free of disease and human suffering.\n\n[PAWN_possessive

## i have huggingface 
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")
and a pandas dataframe with a single column where each row represents sentences please write me code that makes it finetune

In [39]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, AdamW
from tqdm import tqdm
import pandas as pd

# Define a custom dataset class for your sentences
class CustomDataset(Dataset):
    def __init__(self, sentences, tokenizer, max_length=128):
        self.sentences = sentences
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = str(self.sentences[idx])
        inputs = self.tokenizer(sentence, return_tensors="pt", max_length=self.max_length, truncation=True)
        return inputs

# Load your GPT-2 model and tokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2").to(device)

# Load your DataFrame with sentences
# For example, you can load your data like this:
df = pd.read_csv("raw_data/backstory.csv",encoding="unicode_escape")
sentences = df["Desc"].tolist()

# Example sentences (replace this with your actual data)
sentences1 = [
    "This is the first sentence.",
    "Another sentence in your dataset.",
    # Add more sentences as needed
]

# Create a custom dataset
dataset = CustomDataset(sentences, tokenizer)

# Fine-tuning parameters
epochs = 3
learning_rate = 5e-5

# Set up DataLoader
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

# Set up optimizer
optimizer = AdamW(model.parameters(), lr=learning_rate)

# Fine-tune loop
model.train()
for epoch in range(epochs):
    total_loss = 0
    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch + 1}/{epochs}")

    for batch in progress_bar:
        # Move batch to device
        batch = {key: value[0].to(device) for key, value in batch.items()}

        # Forward pass
        outputs = model(**batch, labels=batch["input_ids"])
        loss = outputs.loss

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Update progress bar
        progress_bar.set_postfix({"Loss": total_loss / len(dataloader)})

# Save the fine-tuned model
model.save_pretrained("fine_tuned_gpt2")
tokenizer.save_pretrained("fine_tuned_gpt2")


E:\Anaconda\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 3/3: 100%|██████████| 768/768 [05:55<00:00,  2.16it/s, Loss=nan]  


('fine_tuned_gpt2\\tokenizer_config.json',
 'fine_tuned_gpt2\\special_tokens_map.json',
 'fine_tuned_gpt2\\vocab.json',
 'fine_tuned_gpt2\\merges.txt',
 'fine_tuned_gpt2\\added_tokens.json',
 'fine_tuned_gpt2\\tokenizer.json')

In [50]:
prompt ="In rimworld, "
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

print(input_ids.get_device())


# sample up to 30 tokens
torch.manual_seed(11)
list_out=[]
#['Today I believe we can finally get rid of discrimination," said Rep. Mark Pocan (D-Wis.).\n\n"Just look at the']
for i in range (1):
    for j in range (8):
        outputs = model.generate(input_ids, do_sample=True, max_length=30, temperature =1+0.05*i,top_p=1,repetition_penalty=1.1)
        list_out.append(outputs)


for i in range (1):
    for j in range (8):
        #print("temperature =",11.6+0.75*i,"top_p=",(j+1)/30)
        print(tokenizer.batch_decode(list_out[i*4+j], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

['In rimworld, ikwa or anything not part of the tribe would be confiscated and sold. The group eventually rose into prominence in a violent tribal']
['In rimworld, ikya learned to cook from [PAWN_possessive] father. When [PAWN_pronoun]']
["In rimworld, urchithymic parents often used rituals to enhance [PAWN_possessive] children's intellectual development. In"]
["In rimworld, iced coffee is the only product.\nWhen it comes to making ikki, one doesn't have to eat everything -"]
["In rimworld, ikwa do men wear green leather while shooting. [PAWN_nameDef]'s profession was called to use for guidance"]
['In rimworld, urchin can be a kind-hearted child, and is always polite to befriend others.\n\n[PAWN_name']
['In rimworld, ers are taught to "mix your own food." Traders love this practice - a healthy obsession with plant-based foods.']
["In rimworld, \xa0people just aren't very good at socializing. [PAWN_nameDef] developed an unhealthy aversion to food and"]
